# 第10回 アルゴリズム入門：問題の複雑さ

___
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tsuboshun/begin-python/blob/gh-pages/_sources/workbook/lecture10.ipynb)

___

## この授業で学ぶこと

前回の授業で挿入ソートとクイックソートを紹介し、それらの実行速度に差があることを見た。特にデータ数を増やしたときの実行時間の増え方に大きな差があることを見た。今回は、アルゴリズムの実行時間を見積もるための考え方である計算量について学び、前回の結果について理解を深める。また、それをもとにP・NPという問題の難しさを表す分類について学ぶ。

## 計算量

### 計算量とは

**計算量**（computational complexity）とは、計算ステップ数がデータ数 $N$ に対してどのように増えるかを表す指標である。
計算量はアルゴリズムの実行時間を見積もるために使われる。

例として、次の2つのプログラムの実行時間を考える。

In [ ]:
def single_loop(N):
    count = 0
    for i in range(N):
        count += 1
    return count

In [ ]:
def double_loop(N):
    count = 0
    for i in range(N):
        for j in range(N):
            count += 1
    return count

引数 `N` でループ回数を受け取り、`simple_loop()` は一重ループ、`double_loop()` は二重ループで単純な足し算を実行する。
これらの関数の実行時間を次の `exec_time()` で測定してみよう。

In [ ]:
import time

def exec_time(func, N):
    start = time.time()
    func(N)
    end = time.time()
    return end - start

In [ ]:
N = 10 ** 3
print(f"single_loopの実行時間： {exec_time(single_loop, N)}")
print(f"double_loopの実行時間: {exec_time(double_loop, N)}")

データ数と実行時間の関係を調べてみると、Google Colabの環境では次のようになった（合計の列は後の説明で使う）。

| `N`  |  `single_loop(N)` の実行時間（秒） |  `double_loop(N)` の実行時間（秒） | 合計 |
| ---- | ---- | ---- | ---- |
| 100  |  0.00000429  | 0.000491 | 0.000495 |
| 1000  |  0.0000438  | 0.0296 | 0.0296 |
| 10000  |  0.000513  | 4.00 | 4.00 |
| 100000  |  0.00373  | 344 | 344 | 

$N$ が10倍になると、`single_loop(N)` の実行時間はおよそ10倍、`double_loop(N)` の実行時間はおよそ100倍に増えていることがわかる。

これは次のように理解できる。`single_loop(N)` では `count = 0` を1回実行し、`count += 1` を $N$ 回実行している。$N$ が十分大きいとき、全体の計算ステップ数はほぼ $N$ に比例するので、$N$ が10倍になると実行時間もおよそ10倍になる。一方で、`double_loop(N)` では `count = 0` を1回実行し、`count += 1` を $N^2$ 回実行している。$N$ が十分大きいとき、全体の計算ステップ数はほぼ $N^2$ に比例するので、$N$ が10倍になると実行時間はおよそ100倍になる。

このようなデータ数と計算ステップ数の関係について、 `single_loop(N)` の計算量は $O(N)$、`double_loop(N)` の計算量は $O(N^2)$ であると言う。この記法はランダウ（Landau）のオーダー記法と呼ばれており、例えば $O(N)$ はオーダー$N$と読む。

計算量がわかると、大まかな実行時間の見積もりができる。例えば、`double_loop(N)` は $N = 10^5$ のとき $6$ 分くらいかかっているので、その $10$ 倍の $N = 10^6$ のとき $6\times 10^2 = 600$ 分（つまり $10$ 時間！）くらいかかるだろうと見積もることができる。一般的には、表の結果を踏まえると、計算ステップ数 $10^7\sim10^8$ につき $1$ 秒程度かかると見積もることができる。もちろんこれはプログラムの内容やアルゴリズム、実行環境にも依るので、大まかな見積もりではあるが、実際にプログラムを書く前に実行時間のあたりをつけるのに役立つ。

```{admonition} 発展的な話題： 関数もオブジェクト
:class: note
第4回の授業で「Pythonに出てくるほとんど全ての要素はオブジェクトとして作られている」と説明したが、関数も例外ではない。関数もオブジェクトであり、一種のデータとして扱うことができる。したがって、その他のデータと同じく変数に代入することもできる。
<pre>func = single_loop
func(10**2)  # これはsingle_loop(10**2)と同じ！</pre>
`exec_time()` では引数 `func` で関数を受け取り、`exec_time()` の内部で `func(N)` と実行している。このようなことが可能なのも、関数がオブジェクトとして作られているからである。
```

### オーダー記法について

さて、計算量のオーダー記法について次の2点を補足する。
1. オーダー記法では定数倍の寄与は考えない
2. オーダー記法では低次の項の寄与は考えない  

1つ目については、例えば計算ステップ数が $2N$ であったとしたとしても、オーダー記法としては $O(N)$ と書くということである。
これは定数倍があったとしても、データ数 $N$ が増えたときの計算量の増え方には関係がないからである。つまり、データ数が $N'$ に増えたとして、計算ステップ数の比を考えると $(cN') / (cN) = N' / N$ というように定数倍 $c$ はあってもなくても変わらない。

2つ目については、例えば計算ステップ数が $N^2 + N$ であったとしても、オーダー記法としては $O(N^2)$ と書くということである。
これは実行時間が問題になるような $N$ の大きい領域では、低次の $O(N)$ の寄与はほとんど無視できるからである。
例えば、`single_loop(N)` と `double_loop(N)` の両方を実行するプログラムを考えると、計算ステップ数はおよそ $N^2 + N$ になる。
このプログラムの実行時間は `single_loop(N)` と `double_loop(N)` の実行時間の和になるが、先ほどの表を参照すると、これは `double_loop(N)` の実行時間とほとんど変わらない。

### 関数の比較

In [ ]:
# TODO: log関数の説明など

### ソートアルゴリズムの計算量

それでは、挿入ソートとクイックソートについて計算量を確認してみよう。これらのアルゴリズムでは、`single_loop()` や `double_loop()` と違って、入力データの並び順によって計算ステップ数が大きく変わりうる。そこで最悪なケースにおける計算量を **最悪計算量**、平均的なケースにおける計算量を **平均計算量** として区別して考える。

挿入ソートとクイックソートの計算量は以下の通りである。

|   |  挿入ソート  |  クイックソート | 
| ---- | ---- | ---- |
| 最悪計算量 |  $O(N^2)$  | $O(N^2)$ |
| 平均計算量  |  $O(N^2)$  | $O(N\log N)$ |

$N$ が十分大きいとき、$\log N < N$ の大小関係が成り立つ。したがって、平均計算量についてはクイックソートの方が小さい。
前回の授業で、クイックソートの方が挿入ソートより高速に動作することを見たが、平均計算量の違いが実行時間の差につながっていたわけである。

例えば、これらの計算量から平均的なケースにおいて、挿入ソートは $N = 10^4$ で数秒かかり、クイックソートは $N = 10^6$ で数秒かかると予想できる。なぜなら、先ほど「計算ステップ数 $10^7\sim10^8$ につき $1$ 秒程度かかる」と書いたが、$N = 10^4$ で $N^2 = 10^8$ となり、$N = 10^6$ で $N\log N = 2\times 10^7$ となるからである[^f1]。これらは大まかな見積もりではあるが、前回のノートブック上で実際に測定してみると、大体当たっていることが確かめられる。

[^f1]:対数の底は2として計算した。対数の底に何をとっても定数倍の違いでしかないので、オーダー記法の観点からは底の違いは無視できる。実際、$\log_a N / \log_b N = \log_a b$ より両者の比は $N$ に依らない定数である。

挿入ソートの計算量の求め方を説明する。
`insert_sort()` 関数のfor文に着目する。変数 `i` は1からN-1までを動くが、それぞれの `i` の値に対して内側のwhile文は、最悪のケースで `i` 回実行される（{numref}`insert_sort`の3段目の状況）。したがって、全体の計算ステップ数は

$$
1 + 2 + ... + N-1 = \frac{N(N-1)}{2}
$$

により $N(N-1)/2$ 回と求められる[^f2]。よって、定数倍や低次項を落としてオーダー表記に直すことで、最悪計算量は $O(N^2)$ となる。

[^f2]: `single_loop()` や `double_loop()` のときとは異なり、ここではループ1回を1ステップとして計算ステップ数を求めている。代入などの演算単位で数えたとしても定数倍の違いでしかないので、計算量を求める上ではこれで問題ない。

平均計算量の求め方もほぼ同様である。最悪の場合は、各 `i` につき内側のwhile文が `i` 回実行されると説明したが、平均的にはその半分の回数実行されると考えられる。したがって、全体の平均的な計算ステップ数は、先ほどの結果を2で割った $N(N-1)/4$ 回と求められる。よって、平均計算量は $O(N^2)$ となる。

次にクイックソートの計算量の求め方を説明する。きちんと説明しようとすると少し複雑な数式が出てくるので、ここではざっくりとした直感的な説明を行う。
`quick_sort()` 関数のfor文に着目すると、これは `right - left + 1` 回、つまり {numref}`quick_sort1` の青の領域の長さ分だけ実行される。
したがって、{numref}`quick_sort1` の各段の計算ステップ数は、多めに見積もって $O(N)$ である。
あとは {numref}`quick_sort1` の段数を見積もって掛け算すればよい。

各段における `quick_sort()` の適用範囲の長さの最大値は、最悪のケースでは再帰1回につき1つずつしか減らない（どのような入力データでそうなるだろうか？）。したがって、最も多いケースで {numref}`quick_sort1` の段数は $O(N)$ である。
よって、全体の最悪計算量は $O(N^2)$ となる。

各段における `quick_sort()` の適用範囲の長さの最大値は、最良のケースでは再帰1回につき半分になるので、最も少ないケースではおよそ $O(\log N)$ 段になる。実は平均的なケースでも $O(\log N)$ 段であることを示すことができる。
よって、全体の平均計算量は $O(N\log N)$ となる。

## 問題の難しさ